# NOAA GHCN By-Year Data Exploration

This notebook explores the daily climate observations from the NOAA GHCN dataset, organized by year in Parquet format.

**Data Source:** `s3://noaa-ghcn-pds/parquet/by_year/`

**Structure:** Partitioned by year (e.g., `YEAR=2023/`)

**Key Fields:**
- `ID` - Station identifier
- `DATE` - Observation date (YYYYMMDD)
- `ELEMENT` - Type of observation (TMAX, TMIN, PRCP, etc.)
- `DATA_VALUE` - The observation value
- Quality/source flags

In [ ]:
# Install required packages if needed
# !pip install pandas pyarrow s3fs fsspec

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import s3fs

# S3 filesystem (no credentials needed for public bucket)
S3_BUCKET = 'noaa-ghcn-pds'
PARQUET_PATH = f's3://{S3_BUCKET}/parquet/by_year'
fs = s3fs.S3FileSystem(anon=True)

## 1. Discover Available Years

In [ ]:
# List all available year partitions
year_partitions = fs.ls(f'{S3_BUCKET}/parquet/by_year/')
years = sorted([int(p.split('YEAR=')[1].rstrip('/')) for p in year_partitions if 'YEAR=' in p])

print(f"Available years: {min(years)} to {max(years)}")
print(f"Total years of data: {len(years)}")
print(f"\nFirst 10 years: {years[:10]}")
print(f"Last 10 years: {years[-10:]}")

## 2. Explore Schema and Sample Data

Let's look at a recent year to understand the data structure.

In [ ]:
# Read schema from a sample file
sample_year = 2023
sample_path = f'{S3_BUCKET}/parquet/by_year/YEAR={sample_year}'

# Get parquet files in this partition
parquet_files = fs.ls(sample_path)
print(f"Files in YEAR={sample_year}: {len(parquet_files)}")
print(f"\nFirst file: {parquet_files[0]}")

In [ ]:
# Read schema
with fs.open(parquet_files[0], 'rb') as f:
    schema = pq.read_schema(f)

print("Parquet Schema:")
print("=" * 40)
for field in schema:
    print(f"  {field.name}: {field.type}")

In [ ]:
# Read sample data from one year
df_sample = pd.read_parquet(
    f's3://{sample_path}',
    storage_options={'anon': True}
)

print(f"Shape: {df_sample.shape}")
print(f"\nColumn dtypes:")
print(df_sample.dtypes)
print(f"\nSample data:")
df_sample.head(10)

In [ ]:
# Memory usage
print(f"Memory usage for year {sample_year}:")
print(df_sample.info(memory_usage='deep'))

## 3. Data Element Analysis

In [ ]:
# Count by element type
element_counts = df_sample['ELEMENT'].value_counts()
print(f"Observation counts by element type ({sample_year}):")
print(element_counts.head(20))

In [ ]:
# Element descriptions
element_descriptions = {
    'PRCP': 'Precipitation (tenths of mm)',
    'SNOW': 'Snowfall (mm)',
    'SNWD': 'Snow depth (mm)',
    'TMAX': 'Maximum temperature (tenths of degrees C)',
    'TMIN': 'Minimum temperature (tenths of degrees C)',
    'TAVG': 'Average temperature (tenths of degrees C)',
    'AWND': 'Average daily wind speed (tenths of m/s)',
    'WSFG': 'Peak gust wind speed (tenths of m/s)',
    'WT01': 'Fog, ice fog, or freezing fog',
    'WT02': 'Heavy fog or heavy freezing fog',
    'WT03': 'Thunder',
    'WT04': 'Ice pellets, sleet, snow pellets',
    'WT05': 'Hail',
    'WT06': 'Glaze or rime',
    'WT07': 'Dust, volcanic ash, sand, or spray',
    'WT08': 'Smoke or haze'
}

print("Common element codes and meanings:")
for code, desc in element_descriptions.items():
    print(f"  {code}: {desc}")

## 4. Quality Flags Analysis

In [ ]:
# M_FLAG - Measurement flag
print("M_FLAG (Measurement) distribution:")
print(df_sample['M_FLAG'].value_counts(dropna=False).head(10))

print("\nM_FLAG meanings:")
print("  Blank = no flag")
print("  B = precipitation total from two 12-hour totals")
print("  D = precipitation total from four 6-hour totals")
print("  H = precipitation total from eight 3-hour totals")
print("  L = temperature lagged from previous day")
print("  T = trace of precipitation")

In [ ]:
# Q_FLAG - Quality flag
print("Q_FLAG (Quality) distribution:")
print(df_sample['Q_FLAG'].value_counts(dropna=False).head(10))

print("\nQ_FLAG meanings:")
print("  Blank = passed all quality checks")
print("  D = failed duplicate check")
print("  G = failed gap check")
print("  I = failed internal consistency check")
print("  K = failed streak/frequent-value check")
print("  N = failed naught check")
print("  S = failed spatial consistency check")
print("  X = failed bounds check")

In [ ]:
# S_FLAG - Source flag
print("S_FLAG (Source) distribution:")
print(df_sample['S_FLAG'].value_counts(dropna=False).head(10))

## 5. Station Coverage Analysis

In [ ]:
# Stations reporting in this year
stations_in_year = df_sample['ID'].nunique()
print(f"Unique stations reporting in {sample_year}: {stations_in_year:,}")

# Stations by country (first 2 chars of ID)
df_sample['country'] = df_sample['ID'].str[:2]
stations_by_country = df_sample.groupby('country')['ID'].nunique().sort_values(ascending=False)
print(f"\nTop 15 countries by station count:")
print(stations_by_country.head(15))

In [ ]:
# Observations per station
obs_per_station = df_sample.groupby('ID').size()
print(f"Observations per station statistics:")
print(f"  Min: {obs_per_station.min():,}")
print(f"  Max: {obs_per_station.max():,}")
print(f"  Mean: {obs_per_station.mean():,.0f}")
print(f"  Median: {obs_per_station.median():,.0f}")

## 6. Temperature Data Deep Dive

In [ ]:
# Filter to temperature data only
temp_data = df_sample[df_sample['ELEMENT'].isin(['TMAX', 'TMIN', 'TAVG'])].copy()

# Convert to actual Celsius (divide by 10)
temp_data['temp_celsius'] = temp_data['DATA_VALUE'] / 10

print(f"Temperature observations: {len(temp_data):,}")
print(f"\nTemperature statistics (Celsius):")
temp_data.groupby('ELEMENT')['temp_celsius'].describe()

In [ ]:
# Extreme temperatures
print("Extreme TMAX values (hottest):")
hottest = temp_data[temp_data['ELEMENT'] == 'TMAX'].nlargest(10, 'temp_celsius')[['ID', 'DATE', 'temp_celsius']]
print(hottest)

print("\nExtreme TMIN values (coldest):")
coldest = temp_data[temp_data['ELEMENT'] == 'TMIN'].nsmallest(10, 'temp_celsius')[['ID', 'DATE', 'temp_celsius']]
print(coldest)

## 7. Precipitation Data Deep Dive

In [ ]:
# Filter to precipitation
prcp_data = df_sample[df_sample['ELEMENT'] == 'PRCP'].copy()

# Convert to mm (divide by 10, since stored in tenths of mm)
prcp_data['precip_mm'] = prcp_data['DATA_VALUE'] / 10

print(f"Precipitation observations: {len(prcp_data):,}")
print(f"\nPrecipitation statistics (mm):")
print(prcp_data['precip_mm'].describe())

In [ ]:
# Heaviest rainfall events
print("Top 10 heaviest daily precipitation (mm):")
heaviest_rain = prcp_data.nlargest(10, 'precip_mm')[['ID', 'DATE', 'precip_mm']]
print(heaviest_rain)

## 8. Reading Multiple Years (Efficient Approach)

In [ ]:
# Example: Read multiple years with filtering (more efficient)
# This reads only specific columns and filters to reduce memory

def read_years_filtered(start_year, end_year, elements=['TMAX', 'TMIN', 'PRCP'], columns=None):
    """
    Read multiple years of data with optional filtering.
    
    Parameters:
    - start_year, end_year: Year range to read
    - elements: List of elements to include (None for all)
    - columns: List of columns to read (None for all)
    """
    dfs = []
    
    for year in range(start_year, end_year + 1):
        path = f's3://{S3_BUCKET}/parquet/by_year/YEAR={year}'
        try:
            df = pd.read_parquet(
                path,
                columns=columns,
                storage_options={'anon': True}
            )
            if elements:
                df = df[df['ELEMENT'].isin(elements)]
            df['YEAR'] = year
            dfs.append(df)
            print(f"  Loaded {year}: {len(df):,} rows")
        except Exception as e:
            print(f"  Skipped {year}: {e}")
    
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

# Example: Read last 3 years of temperature data
# Uncomment to run (may take a few minutes)
# recent_temps = read_years_filtered(2021, 2023, elements=['TMAX', 'TMIN'], columns=['ID', 'DATE', 'ELEMENT', 'DATA_VALUE'])
# print(f"\nTotal rows: {len(recent_temps):,}")

## 9. Estimate Data Size for Snowflake

Estimate total data volume to plan Snowflake loading.

In [ ]:
# Check file sizes for a few years
def get_partition_size(year):
    """Get total size of parquet files for a year in MB."""
    path = f'{S3_BUCKET}/parquet/by_year/YEAR={year}'
    try:
        files = fs.ls(path, detail=True)
        total_bytes = sum(f['size'] for f in files)
        return total_bytes / (1024 * 1024)  # MB
    except:
        return 0

# Sample years to estimate
sample_years_for_size = [1900, 1950, 1980, 2000, 2010, 2020, 2023]
print("Parquet file sizes by year:")
sizes = {}
for year in sample_years_for_size:
    size = get_partition_size(year)
    sizes[year] = size
    print(f"  {year}: {size:.1f} MB")

In [ ]:
# Row count estimate
print(f"\nRow count for sample year ({sample_year}): {len(df_sample):,}")
print(f"\nEstimated total rows (rough): ~{len(df_sample) * len(years):,}")
print("\n(Actual total will vary as older years have fewer observations)")

## 10. Summary for Snowflake Loading

In [ ]:
print("=" * 60)
print("SUMMARY FOR SNOWFLAKE LOADING")
print("=" * 60)
print(f"\nData Location: s3://noaa-ghcn-pds/parquet/by_year/")
print(f"Format: Parquet (partitioned by YEAR)")
print(f"Years available: {min(years)} to {max(years)} ({len(years)} years)")
print(f"\nSchema:")
for field in schema:
    print(f"  {field.name}: {field.type}")
print(f"\nSample year ({sample_year}):")
print(f"  Rows: {len(df_sample):,}")
print(f"  Unique stations: {df_sample['ID'].nunique():,}")
print(f"  Unique elements: {df_sample['ELEMENT'].nunique()}")
print(f"\nRecommended Snowflake approach:")
print("  1. Create external stage pointing to S3 bucket")
print("  2. Use COPY INTO with Parquet format")
print("  3. Consider partitioning table by YEAR")
print("  4. Create clustering on (ELEMENT, ID) for query performance")

In [ ]:
# Clean up
del df_sample
print("Cleanup complete.")